In [0]:
from google.colab import drive
import numpy as np
import pandas as pd
drive.mount('/content/gdrive')
bengali_news_after_preprocessing = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/Bengali_Text_after_preprocessing.pkl')
from sklearn.externals import joblib
filename = '/content/gdrive/My Drive/Projects/Bengali Text Classification/fastText_Bangla_content_full.sav'
loaded_model = joblib.load(filename)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
import keras.backend as K
import numpy as np
number_of_sample, max_number_of_words, word_vector_size = 40000, 50, 32
temp = bengali_news_after_preprocessing.loc[:number_of_sample-1,:max_number_of_words-1]

Using TensorFlow backend.


In [0]:
temp = temp.replace(['ঘস', 'ফগ', 'ঝবঃ', 'ঋন', 'ঊঘ', '\u09e4', 'ওৎ', 'গথ', 'খঢ', 'ঝ’', ' ং', 'ঔ', 'ডড', 'গঘ'], None)
X = np.zeros((number_of_sample, max_number_of_words, word_vector_size), dtype=K.floatx())
for i in temp.index:
  X[i,:,:] = loaded_model.wv[temp.loc[i,:]]

In [0]:
bengali_news = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/40k_bangla_newspaper_article.p')
bengali_news_dataframe = pd.DataFrame(bengali_news)
y = bengali_news_dataframe['category']

In [0]:
from sklearn import preprocessing
import keras
import numpy as np
le = preprocessing.LabelEncoder()
le.fit(y)
enc = le.transform(y)
y = keras.utils.to_categorical(enc)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle = True, test_size=0.125)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, shuffle = True, test_size=0.20)

In [0]:
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard


model = Sequential()

model.add(Bidirectional(LSTM(512, dropout=0.2, recurrent_dropout=0.3), input_shape=(max_number_of_words, word_vector_size)))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))

model.add(Dense(13, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_10 (Bidirectio (None, 1024)              2232320   
_________________________________________________________________
dense_9 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
__________

In [0]:
history = model.fit(X_train, y_train, batch_size= 500, shuffle=True, epochs= 100, validation_data=(X_val, y_val))

Train on 35000 samples, validate on 4000 samples
Epoch 1/100
35000/35000 [==============================] - 35s 1ms/step - loss: 1.6602 - acc: 0.4698 - val_loss: 1.3901 - val_acc: 0.6315
Epoch 2/100
35000/35000 [==============================] - 35s 1ms/step - loss: 1.4003 - acc: 0.6015 - val_loss: 1.2061 - val_acc: 0.6525
Epoch 3/100
35000/35000 [==============================] - 35s 999us/step - loss: 1.2590 - acc: 0.6364 - val_loss: 1.0996 - val_acc: 0.6610
Epoch 4/100
35000/35000 [==============================] - 35s 999us/step - loss: 1.1679 - acc: 0.6548 - val_loss: 1.0163 - val_acc: 0.6918
Epoch 5/100
35000/35000 [==============================] - 35s 1000us/step - loss: 1.0893 - acc: 0.6851 - val_loss: 0.9432 - val_acc: 0.7470
Epoch 6/100
35000/35000 [==============================] - 35s 998us/step - loss: 1.0251 - acc: 0.7078 - val_loss: 0.8896 - val_acc: 0.7488
Epoch 7/100
35000/35000 [==============================] - 35s 1ms/step - loss: 0.9718 - acc: 0.7213 - val_loss: 0

In [0]:
predicts = model.predict(X_test)
import numpy as np
def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [0]:
from sklearn import metrics

print(metrics.accuracy_score(y_test, y_preds))

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

0.842
[[  4   1   0   0   0   1   0   0   0   3   1   0]
 [  0 251   0  15   4   1   5   3   0  24   0   1]
 [  0   0   0   0   0   1   1   0   0   2   0   0]
 [  0  12   0  86   0   1   1   0   0  10   0   2]
 [  0   7   0   1  16   1   0   0   0   1   0   0]
 [  1   0   0   0   0  59   1   2   0   3   1   2]
 [  0   2   0   0   0   0  43   0   0   3   0   0]
 [  1   1   0   1   1   0   0  20   0   2   0   2]
 [  0   0   0   0   0   0   1   0   0   1   0   0]
 [  2  12   0   3   1   2   3   2   0 252   1   1]
 [  1   1   0   0   0   0   1   0   0   1  69   0]
 [  0   0   0   1   0   0   0   2   0   0   0  42]]
                    precision    recall  f1-score   support

art-and-literature       0.44      0.40      0.42        10
        bangladesh       0.87      0.83      0.85       304
       durporobash       0.00      0.00      0.00         4
           economy       0.80      0.77      0.79       112
         education       0.73      0.62      0.67        26
     entertainment  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
accuracy, val_accuracy = np.array(history.history["acc"]), np.array(history.history["val_acc"])
accuracy, val_accuracy = accuracy.reshape(100,1), val_accuracy.reshape(100,1)
accuracies = np.concatenate((accuracy,val_accuracy),axis=1)
np.savetxt('/content/gdrive/My Drive/Projects/Bengali Text Classification/temp.csv',accuracies,delimiter=",")